In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
# def fix_nan_train(data):
#     # 使用对应分类的列均值来填充nan
#     real_some = data.loc[data.values[:,-1] == 1]
#     fake_some = data.loc[data.values[:,-1] == 0]
#     real_some_mod = real_some.fillna(real_some.mean(axis=0))
#     fake_some_mod = fake_some.fillna(fake_some.mean(axis=0))
#     data_mod = pd.concat([real_some_mod, fake_some_mod], axis=0).sort_index()
#     return data_mod

# def fix_nan_test(data):
#     return data.fillna(data.mean(axis=0))
    
def fix_nan_all(data):
    return data.fillna(1)

def flatten(data):
    data = data + data.min(axis=1).reshape((-1,1))
    return data / data.max(axis=1).reshape((-1,1))
    
def load_file(file, method):
    data = pd.read_csv(file, header=None)
    values = fix_nan_all(data).values
    flattened = flatten(values[:, :-1].T)
    return np.row_stack((flattened,np.ones(flattened.shape[1]))), values[:, -1].reshape((1,-1))

In [3]:
train_data, train_label = load_file("salted_fish/salted_fish_train.csv", 'train')
test_data, test_label = load_file("salted_fish/salted_fish_test.csv", 'test')

In [4]:
train_data = train_data.T
test_data = test_data.T

In [5]:
train_label = train_label.reshape((-1,))
test_label = test_label.reshape((-1,))

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(train_data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
sc.mean_.shape

(9,)

In [8]:
train_data_std = sc.fit(train_data)
test_data_std = sc.fit(test_data)

In [9]:
lr = LogisticRegression(C=1.0, penalty='l2', max_iter=100000, tol=0.009)

In [10]:
lr.fit(train_data, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100000, multi_class='ovr',
          n_jobs=1, penalty='l2', random_state=None, solver='liblinear',
          tol=0.009, verbose=0, warm_start=False)

In [11]:
test_prediction = lr.predict(test_data)
test_score = lr.score(test_data, test_label)

In [12]:
print(classification_report(test_label, test_prediction))

             precision    recall  f1-score   support

        0.0       0.78      0.92      0.84       178
        1.0       0.74      0.48      0.58        90

avg / total       0.76      0.77      0.75       268



In [13]:
test_score

0.7686567164179104